In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [15]:
prompt = ChatPromptTemplate.from_template(
    "write {number} poems about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [16]:
chain = prompt | model | output_parser

In [17]:
result = chain.invoke({"number": "2", "topic": "nature"})

In [18]:
result

"1. \n\nIn the forests wild and free,\nNature's beauty I can see.\nTall trees reaching for the sky,\nBirds soaring, singing high.\n\nThe gentle rustle of the leaves,\nThe whisper of the breeze.\nThe sun shining through the canopy,\nNature's wonders never cease to amaze me.\n\nThe scent of flowers in the air,\nThe sound of a distant bear.\nNature's symphony all around,\nIn this peaceful place, I am found.\n\nSo let me wander through the trees,\nAnd feel the earth beneath my knees.\nIn nature's embrace, I am whole,\nConnected to the world, body and soul.\n\n2.\n\nThe ocean's waves crash against the shore,\nA symphony of sound that I adore.\nThe salty air, the seagulls cry,\nNature's beauty never passes by.\n\nThe sun setting on the horizon,\nPainting the sky with colors so vibrant.\nThe water glistening in the light,\nA breathtaking sight, pure and bright.\n\nThe sand beneath my feet, warm and soft,\nA reminder of nature's love, never lost.\nThe rhythm of the tide, ebbing and flowing,\nA

In [50]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [51]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["John studies engineering", "John works three part-time jobs", "John watches TV shows in his free time"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [52]:
retriever.get_relevant_documents("what does John study?")

[Document(page_content='John studies engineering'),
 Document(page_content='John works three part-time jobs'),
 Document(page_content='John watches TV shows in his free time')]

In [53]:
retriever.get_relevant_documents("How many part-time jobs does John work at?")

[Document(page_content='John works three part-time jobs'),
 Document(page_content='John watches TV shows in his free time'),
 Document(page_content='John studies engineering')]

In [54]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [55]:
from langchain.schema.runnable import RunnableMap

In [56]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [58]:
chain.invoke({"question": "How many part-time jobs does John handle?"})

'John works three part-time jobs.'

In [42]:
chain.invoke({"question": "What is John's hobby?"})

'Watching TV shows in his free time'